In [1]:
#import singleton
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS
import read_data as importLib

LESSON_DATA_FOLDER = importLib.LESSON_DATA_FOLDER

In [ ]:
#putil = ParallelUtil('./data/parallel.config.tsv')

In [4]:
merged_names = importLib.readFile('merged.names.principals.tsv')
merged_names.info()

Start: 2018-03-05 16:52:12


/home/paperspace/anaconda3/envs/imdb-env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


End: 2018-03-05 16:56:05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27172692 entries, 0 to 27172691
Data columns (total 13 columns):
tconst               object
ordering             int32
nconst               object
category             object
job                  object
characters           object
primaryName          object
birthYear            int64
deathYear            int64
primaryProfession    object
knownForTitles       object
isAlive              bool
wiki                 object
dtypes: bool(1), int32(1), int64(2), object(9)
memory usage: 2.4+ GB


In [5]:
merged_names.head()

,tconst,ordering,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,isAlive,wiki
0,tt0000001,1,nm1588970,self,NaN,"[""Herself""]",Carmencita,0,0,soundtrack,"tt0057728,tt0000001",True,https://en.wikipedia.org/wiki/Carmencita
1,tt0000001,2,nm0005690,director,NaN,"[""Herself""]",William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt0241763,tt0361921,tt1428455,tt0229235",False,https://en.wikipedia.org/wiki/William_K.L._Dic...
2,tt0000001,3,nm0374658,cinematographer,director of photography,"[""Herself""]",William Heise,0,1910,"cinematographer,director,producer","tt0154152,tt0229235,tt0219560,tt0361921",False,https://en.wikipedia.org/wiki/William_Heise
3,tt0000002,1,nm0721526,director,director of photography,"[""Herself""]",Émile Reynaud,1844,1918,director,"tt0000004,tt0000015,tt0000003,tt0000002",False,https://en.wikipedia.org/wiki/Émile_Reynaud
4,tt0000002,2,nm1335271,composer,director of photography,"[""Herself""]",Gaston Paulin,1839,1903,composer,"tt0000004,tt0000002,tt0000003",False,https://en.wikipedia.org/wiki/Gaston_Paulin


In [17]:
merged_names.head(50).groupby('tconst')['tconst','nconst','ordering','category',
                                        'job','characters','primaryName','birthYear','deathYear',
                                        'wiki','isAlive'
                                       ].count()

,tconst,nconst,ordering,category,job,characters,primaryName,birthYear,deathYear,wiki,isAlive
tconst,,,,,,,,,,,
tt0000001,3,3,3,3,1,3,3,3,3,3,3
tt0000002,2,2,2,2,2,2,2,2,2,2,2
tt0000003,4,4,4,4,4,4,4,4,4,4,4
tt0000004,2,2,2,2,2,2,2,2,2,2,2
tt0000005,3,3,3,3,3,3,3,3,3,3,3
tt0000006,1,1,1,1,1,1,1,1,1,1,1
tt0000007,4,4,4,4,4,4,4,4,4,4,4
tt0000008,3,3,3,3,3,3,3,3,3,3,3
tt0000009,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
merged_names = importLib.readFile('merged.names.principals.tsv')
merged_names.info()